# The Battle of Neighbourhoods

## A description of the problem

#### In this project we intend to compare the neighborhoods of some of the world's largest cities and try to segregate them based on similarities between them. For example, New York City and New Delhi are known to be multi cultural and house people from different backgrounds. This has led to an increase in number of multi cuisine restaurants and cafes. Similarly, both the cities have large number of financial corporations, parks, movie theatres, party halls etc. which make them alike. The intent of this analysis is to find clusters of such cities that are more similar to each other as compared to others.

## Background

#### This is an interesting problem to solve due to increasing number of people migrating between cities nowadays. This analysis will help people explore cities with similar neighborhoods and consequently make migration easy and stressfree. It takes a lot of research and effort for anyone to settle in a new city. This analysis will help such people to choose their target city from those that are similar to their current city.

## Description of Data

#### We will use the Foursquare API to explore neighborhoods in various metropolitan cities of the world. We will use the explore function to get the most common venue categories in each city, and then use this feature to group the cities into clusters. The data set that we get though the explore feature of Foursquare API consists of venue details in the vicinity of a location (latitude and longitude). Venue details comprise of Name, location, category and other important details. We filter top venues near the location (or in that city) and use this data to find similarity between cities. Firstly, data is sourced for world's largest metropolitan cities from citymayors.com Then using geocoder library and Foursquare API, the rest of the analysis is performed.

## Methodology

#### The file containing cities data is read and location data is extracted using geocoders library. Using this location data, top 100 venues of each city are fetched using the Foursquare API. Then the data is rolled up on city level and unique venue categories are determined by applying pandas operation. For each city the top 20 venu categories are selected and K-means clustering algoritm is applied to divide these cities into 5 clusters.

## Results

#### The 49 cities get divided into 5 clusters using the k-means clustering algorithm. It is observed that SanFrancisco itself forms a seperate cluster and similarly Dallas forms a seperate cluster. Out of 49 cities, 34 fall in a single cluster - Cluster 2. These cities are some of the largest cities of the world.

## Discussion

#### It is observed that the model can be improved by (i) Increasing the radius and capturing more venu categories; (ii) Capturing data points like per capita income, population, pollution etc. which are not readily available on foursquare and; (iii) Applying various other clustering techniques.

## Conclusion

#### As per the problem statement a list of 49 cities was successfully analysed to figure out similarities. This analysis will help in determinig which cities are similar to each other.

In [6]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
import folium

In [17]:
cityCoordinates=[]
with open('cities.txt', 'r') as md:
    for line in md:
        address=str(line)
        geolocator = Nominatim(user_agent="ny_explorer")
        try:
            location = geolocator.geocode(address)
            latitude = location.latitude
            longitude = location.longitude
            cityCoordinates.append([address,latitude,longitude])
        except:
            pass
        #print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        

In [18]:
cityCoordinates[:5]

[['Tokyo, Japan\n', 35.6828387, 139.7594549],
 ['New York, USA\n', 40.7127281, -74.0060152],
 ['Los Angeles, USA\n', 34.0536909, -118.242766],
 ['London, UK\n', 51.5073219, -0.1276474],
 ['Chicago, USA\n', 41.8755616, -87.6244212]]

In [19]:
cities_df = pd.DataFrame(cityCoordinates, columns = ['Name', 'Latitude','Longitude']) 
cities_df = cities_df.replace('\n','', regex=True)
cities_df.head()

,Name,Latitude,Longitude
0,"Tokyo, Japan",35.682839,139.759455
1,"New York, USA",40.712728,-74.006015
2,"Los Angeles, USA",34.053691,-118.242766
3,"London, UK",51.507322,-0.127647
4,"Chicago, USA",41.875562,-87.624421


In [20]:
cities_df.shape

(49, 3)

In [21]:
CLIENT_ID = 'P5KWCWJQ0HFPLSQLT44Q0JKEO4AUQ4SLIEDYLOKCMHIR1GC4' # Foursquare ID
CLIENT_SECRET = 'RWUMFK3AHJVVY0SNVQQXHB3GTXDDDWI1AKEHJBHHCPTE0YT0' # Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
radius = 1000 

print('Credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Credentails:
CLIENT_ID: P5KWCWJQ0HFPLSQLT44Q0JKEO4AUQ4SLIEDYLOKCMHIR1GC4
CLIENT_SECRET:RWUMFK3AHJVVY0SNVQQXHB3GTXDDDWI1AKEHJBHHCPTE0YT0


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
# type your answer here
city_venues = getNearbyVenues(names=cities_df['Name'],
                                   latitudes=cities_df['Latitude'],
                                   longitudes=cities_df['Longitude']
                                  )

Tokyo, Japan
New York, USA
Los Angeles, USA
London, UK
Chicago, USA
Paris, France
Mexico City, Mexico
Philadelphia, USA
Osaka/Kobe, Japan
Washington DC, USA
Buenos Aires, Argentina
Boston, USA
Sao Paulo, Brazil
Hong Kong, China
Dallas/Fort Worth, USA
Shanghai, China
Seoul, South Korea
Atlanta, USA
San Francisco/Oakland, USA
Houston, USA
Miami, USA
Toronto, Canada
Moscow, Russia
Mumbai, India
Madrid, Spain
Detroit, USA
Istanbul, Turkey
Seattle, USA
Beijing, China
Metro Manila, Philippines
Rio de Janeiro, Brazil
Sydney, Australia
Jakarta, Indonesia
Delhi, India
Phoenix, USA
Guangzhou, China
Minneapolis, USA
Kolkata, India
San Diego, USA
Singapore, Singapore
Cairo, Egypt
Barcelona, Spain
Melbourne, Australia
Denver, USA
Rome, Italy
Bangkok, Thailand
Milan, Italy
Tehran, Iran
Riyadh, Saudi Arabia


In [24]:
print(city_venues.shape)
city_venues.head()

(2700, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Tokyo, Japan",35.682839,139.759455,Grand Kitchen,35.684026,139.761426,American Restaurant
1,"Tokyo, Japan",35.682839,139.759455,Palace Hotel Tokyo (パレスホテル東京),35.684644,139.761302,Hotel
2,"Tokyo, Japan",35.682839,139.759455,Sushi Kanesaka (鮨 かねさか),35.684559,139.761064,Sushi Restaurant
3,"Tokyo, Japan",35.682839,139.759455,Wadakura Fountain Park (和田倉噴水公園),35.683453,139.760829,Park
4,"Tokyo, Japan",35.682839,139.759455,Amber Palace (琥珀宮),35.684647,139.761458,Chinese Restaurant


In [25]:
city_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Atlanta, USA",8,8,8,8,8,8
"Bangkok, Thailand",25,25,25,25,25,25
"Barcelona, Spain",100,100,100,100,100,100
"Beijing, China",9,9,9,9,9,9
"Boston, USA",100,100,100,100,100,100
"Buenos Aires, Argentina",26,26,26,26,26,26
"Cairo, Egypt",75,75,75,75,75,75
"Chicago, USA",100,100,100,100,100,100
"Dallas/Fort Worth, USA",3,3,3,3,3,3


In [26]:
print('There are {} uniques categories.'.format(len(city_venues['Venue Category'].unique())))

There are 340 uniques categories.


In [27]:
# one hot encoding
city_onehot = pd.get_dummies(city_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
city_onehot['Neighborhood'] = city_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [city_onehot.columns[-1]] + list(city_onehot.columns[:-1])
city_onehot = city_onehot[fixed_columns]

city_onehot.head()

,Zoo,ATM,Accessories Store,African Restaurant,Airport Service,Alsatian Restaurant,American Restaurant,Amphitheater,Antique Shop,Arepa Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
city_grouped = city_onehot.groupby('Neighborhood').mean().reset_index()
city_grouped

,Neighborhood,Zoo,ATM,Accessories Store,African Restaurant,Airport Service,Alsatian Restaurant,American Restaurant,Amphitheater,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Atlanta, USA",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000
1,"Bangkok, Thailand",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000
2,"Barcelona, Spain",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.020000,0.01,0.000000,0.01,0.010000
3,"Beijing, China",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000
4,"Boston, USA",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,...,0.010000,0.000000,0.000000,0.00,0.000000,0.000000,0.02,0.000000,0.00,0.020000
5,"Buenos Aires, Argentina",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.038462,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000
6,"Cairo, Egypt",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000
7,"Chicago, USA",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,...,0.010000,0.000000,0.000000,0.00,0.010000,0.000000,0.00,0.000000,0.00,0.000000
8,"Dallas/Fort Worth, USA",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000
9,"Denver, USA",0.000000,0.016129,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.016129,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.032258


In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = city_grouped['Neighborhood']

for ind in np.arange(city_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(city_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,"Atlanta, USA",Breakfast Spot,College Basketball Court,Event Space,Music Venue,Diner,Art Gallery,Caribbean Restaurant,Intersection,Filipino Restaurant,...,French Restaurant,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food Truck,Food Court,Flower Shop,Fountain
1,"Bangkok, Thailand",Café,History Museum,Buddhist Temple,Palace,Coffee Shop,Dessert Shop,Museum,Noodle House,Bakery,...,Shopping Mall,Chinese Restaurant,Record Shop,Soup Place,Theater,Japanese Restaurant,Spiritual Center,Historic Site,Falafel Restaurant,Fabric Shop
2,"Barcelona, Spain",Tapas Restaurant,Italian Restaurant,Plaza,Bar,Ice Cream Shop,Spanish Restaurant,Mediterranean Restaurant,Coffee Shop,Japanese Restaurant,...,Dessert Shop,Ramen Restaurant,Sandwich Place,Pizza Place,Clothing Store,Mexican Restaurant,Wine Bar,Cocktail Bar,Market,Camera Store
3,"Beijing, China",Historic Site,Park,Concert Hall,Metro Station,Plaza,History Museum,Scenic Lookout,Fabric Shop,Empanada Restaurant,...,Escape Room,Event Space,Exhibit,Yoga Studio,Falafel Restaurant,Electronics Store,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop
4,"Boston, USA",Italian Restaurant,Coffee Shop,Historic Site,Seafood Restaurant,Hotel,Pub,Sandwich Place,Bakery,Park,...,Irish Pub,Pizza Place,Restaurant,Salad Place,Steakhouse,Yoga Studio,Wine Shop,Mediterranean Restaurant,Fish Market,Boutique
5,"Buenos Aires, Argentina",Argentinian Restaurant,Bakery,Pizza Place,Coffee Shop,Ice Cream Shop,Bike Rental / Bike Share,Supermarket,Park,Fast Food Restaurant,...,Café,Amphitheater,Dance Studio,Science Museum,Gym,Shop & Service,Fabric Shop,Exhibit,Event Space,Yoga Studio
6,"Cairo, Egypt",Café,Hotel,Falafel Restaurant,Lounge,Italian Restaurant,Beer Garden,Plaza,Egyptian Restaurant,Fast Food Restaurant,...,Casino,History Museum,Bookstore,Pizza Place,Theater,Kebab Restaurant,Hotel Pool,Chocolate Shop,Spa,French Restaurant
7,"Chicago, USA",Coffee Shop,Sandwich Place,Hotel,Pizza Place,Bookstore,Pub,Museum,Theater,Arts & Crafts Store,...,Boutique,Fountain,Music Venue,Café,Asian Restaurant,Trail,Garden,Art Museum,Dance Studio,Hot Dog Joint
8,"Dallas/Fort Worth, USA",Seafood Restaurant,Mexican Restaurant,Hotel,Yoga Studio,Empanada Restaurant,Escape Room,Event Space,Exhibit,Fabric Shop,...,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck
9,"Denver, USA",Sandwich Place,Coffee Shop,Asian Restaurant,Mexican Restaurant,Yoga Studio,Gym,Pub,Exhibit,Noodle House,...,Art Gallery,Restaurant,Café,Organic Grocery,Chinese Restaurant,Middle Eastern Restaurant,Outdoor Sculpture,Lounge,Marijuana Dispensary,Bookstore


In [31]:
# set number of clusters
kclusters = 5

city_grouped_clustering = city_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 1, 0, 2, 2, 2, 2, 4, 2])

In [43]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

city_merged = cities_df
city_merged=city_merged.rename(columns={"Name": "Neighborhood"})

city_merged = city_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

city_merged.head() # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,"Tokyo, Japan",35.682839,139.759455,2.0,Historic Site,Japanese Restaurant,Italian Restaurant,Café,Convenience Store,Chinese Restaurant,...,Clothing Store,Lounge,Sake Bar,Business Center,Hotel Bar,Bridge,Supermarket,Sukiyaki Restaurant,Steakhouse,Lake
1,"New York, USA",40.712728,-74.006015,2.0,Coffee Shop,Sandwich Place,Hotel,Falafel Restaurant,Burger Joint,Pizza Place,...,French Restaurant,Park,Bakery,Plaza,Gym / Fitness Center,Cocktail Bar,Salad Place,Hotel Bar,Bubble Tea Shop,Cuban Restaurant
2,"Los Angeles, USA",34.053691,-118.242766,2.0,Museum,Japanese Restaurant,Coffee Shop,Noodle House,Music Venue,Speakeasy,...,Supermarket,Men's Store,Sushi Restaurant,Sculpture Garden,Candy Store,Food & Drink Shop,Gift Shop,Sandwich Place,Jazz Club,Theater
3,"London, UK",51.507322,-0.127647,2.0,Theater,Hotel,Plaza,Cocktail Bar,Japanese Restaurant,Sandwich Place,...,Steakhouse,Pub,Café,Art Gallery,Art Museum,Burger Joint,Tea Room,English Restaurant,Ramen Restaurant,Boutique
4,"Chicago, USA",41.875562,-87.624421,2.0,Coffee Shop,Sandwich Place,Hotel,Pizza Place,Bookstore,Pub,...,Boutique,Fountain,Music Venue,Café,Asian Restaurant,Trail,Garden,Art Museum,Dance Studio,Hot Dog Joint


In [48]:
city_merged.loc[city_merged['Cluster Labels'] == 0, city_merged.columns[[0] + list(range(4, city_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
22,"Moscow, Russia",Plaza,Palace,History Museum,Historic Site,Boutique,Concert Hall,Park,Ukrainian Restaurant,Church,...,Italian Restaurant,Government Building,Hostel,Skating Rink,English Restaurant,Exhibit,Escape Room,Yoga Studio,Fabric Shop,Empanada Restaurant
28,"Beijing, China",Historic Site,Park,Concert Hall,Metro Station,Plaza,History Museum,Scenic Lookout,Fabric Shop,Empanada Restaurant,...,Escape Room,Event Space,Exhibit,Yoga Studio,Falafel Restaurant,Electronics Store,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop
44,"Rome, Italy",Historic Site,Hotel,Temple,Monument / Landmark,Pub,Italian Restaurant,Fountain,History Museum,Pizza Place,...,Plaza,Sandwich Place,Road,Dive Bar,Bar,Tourist Information Center,Theater,Roman Restaurant,Restaurant,Market


In [49]:
city_merged.loc[city_merged['Cluster Labels'] == 1, city_merged.columns[[0] + list(range(4, city_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
17,"Atlanta, USA",Breakfast Spot,College Basketball Court,Event Space,Music Venue,Diner,Art Gallery,Caribbean Restaurant,Intersection,Filipino Restaurant,...,French Restaurant,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food Truck,Food Court,Flower Shop,Fountain
20,"Miami, USA",Italian Restaurant,Lounge,Pharmacy,Cocktail Bar,Coffee Shop,Hotel,Peruvian Restaurant,Sandwich Place,Print Shop,...,Tapas Restaurant,Salad Place,Gluten-free Restaurant,Salon / Barbershop,Scenic Lookout,Bistro,Theater,Seafood Restaurant,Event Space,French Restaurant
21,"Toronto, Canada",Clothing Store,Coffee Shop,Electronics Store,Plaza,Cosmetics Shop,Theater,Diner,Thai Restaurant,New American Restaurant,...,Restaurant,Chinese Restaurant,Pizza Place,Ramen Restaurant,Salad Place,Café,Seafood Restaurant,Cocktail Bar,Shoe Store,Shopping Mall
25,"Detroit, USA",Coffee Shop,American Restaurant,New American Restaurant,Steakhouse,Restaurant,Cocktail Bar,Bar,Café,Diner,...,Sandwich Place,Hotel,Outdoor Sculpture,Lounge,Mexican Restaurant,Hot Dog Joint,Speakeasy,Dive Bar,Sporting Goods Shop,Plaza
30,"Rio de Janeiro, Brazil",Bar,Restaurant,Brazilian Restaurant,Café,Train Station,Department Store,Supermarket,Middle Eastern Restaurant,Gym,...,Samba School,Pharmacy,Fried Chicken Joint,Hotel,Ice Cream Shop,Plaza,Food Truck,Diner,Cultural Center,Food & Drink Shop
31,"Sydney, Australia",Theater,Cocktail Bar,Australian Restaurant,Concert Hall,Plaza,Opera House,Café,Park,Fondue Restaurant,...,Fish Market,Filipino Restaurant,English Restaurant,Fast Food Restaurant,Food,Farmers Market,Food & Drink Shop,Falafel Restaurant,Fabric Shop,Exhibit
34,"Phoenix, USA",Hotel,Coffee Shop,American Restaurant,Lounge,Pizza Place,Basketball Stadium,Salon / Barbershop,Cocktail Bar,Bar,...,Rock Club,Theater,Shipping Store,Mexican Restaurant,Fried Chicken Joint,Basketball Court,Sports Bar,New American Restaurant,Southern / Soul Food Restaurant,Opera House
41,"Barcelona, Spain",Tapas Restaurant,Italian Restaurant,Plaza,Bar,Ice Cream Shop,Spanish Restaurant,Mediterranean Restaurant,Coffee Shop,Japanese Restaurant,...,Dessert Shop,Ramen Restaurant,Sandwich Place,Pizza Place,Clothing Store,Mexican Restaurant,Wine Bar,Cocktail Bar,Market,Camera Store
42,"Melbourne, Australia",Coffee Shop,Café,Bar,Cocktail Bar,Shopping Mall,Dessert Shop,Italian Restaurant,Clothing Store,Movie Theater,...,Candy Store,Pizza Place,Cosmetics Shop,Juice Bar,Miscellaneous Shop,Dumpling Restaurant,Shopping Plaza,Hobby Shop,Burger Joint,Scandinavian Restaurant


In [50]:
city_merged.loc[city_merged['Cluster Labels'] == 2, city_merged.columns[[0] + list(range(4, city_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,"Tokyo, Japan",Historic Site,Japanese Restaurant,Italian Restaurant,Café,Convenience Store,Chinese Restaurant,Park,French Restaurant,Asian Restaurant,...,Clothing Store,Lounge,Sake Bar,Business Center,Hotel Bar,Bridge,Supermarket,Sukiyaki Restaurant,Steakhouse,Lake
1,"New York, USA",Coffee Shop,Sandwich Place,Hotel,Falafel Restaurant,Burger Joint,Pizza Place,American Restaurant,Spa,Gym,...,French Restaurant,Park,Bakery,Plaza,Gym / Fitness Center,Cocktail Bar,Salad Place,Hotel Bar,Bubble Tea Shop,Cuban Restaurant
2,"Los Angeles, USA",Museum,Japanese Restaurant,Coffee Shop,Noodle House,Music Venue,Speakeasy,Shopping Mall,Bookstore,Breakfast Spot,...,Supermarket,Men's Store,Sushi Restaurant,Sculpture Garden,Candy Store,Food & Drink Shop,Gift Shop,Sandwich Place,Jazz Club,Theater
3,"London, UK",Theater,Hotel,Plaza,Cocktail Bar,Japanese Restaurant,Sandwich Place,Garden,Bakery,Monument / Landmark,...,Steakhouse,Pub,Café,Art Gallery,Art Museum,Burger Joint,Tea Room,English Restaurant,Ramen Restaurant,Boutique
4,"Chicago, USA",Coffee Shop,Sandwich Place,Hotel,Pizza Place,Bookstore,Pub,Museum,Theater,Arts & Crafts Store,...,Boutique,Fountain,Music Venue,Café,Asian Restaurant,Trail,Garden,Art Museum,Dance Studio,Hot Dog Joint
5,"Paris, France",French Restaurant,Ice Cream Shop,Plaza,Historic Site,Creperie,Park,Italian Restaurant,Art Gallery,Bookstore,...,Restaurant,Clothing Store,Tea Room,Coffee Shop,Bakery,Pedestrian Plaza,Pastry Shop,Hotel,Wine Bar,Dessert Shop
6,"Mexico City, Mexico",Arts & Crafts Store,Art Museum,Ice Cream Shop,Museum,Clothing Store,Boutique,Restaurant,Hotel,Historic Site,...,Bakery,Department Store,Concert Hall,Plaza,Seafood Restaurant,Bar,Coffee Shop,Taco Place,Bookstore,Café
7,"Philadelphia, USA",Hotel,Bakery,Coffee Shop,Dessert Shop,Clothing Store,Israeli Restaurant,Chinese Restaurant,Pub,Mediterranean Restaurant,...,Sandwich Place,Spa,Sushi Restaurant,Taco Place,Deli / Bodega,Seafood Restaurant,Mexican Restaurant,Plaza,Grocery Store,American Restaurant
8,"Osaka/Kobe, Japan",Café,French Restaurant,Chinese Restaurant,Intersection,Hotel,Convenience Store,Park,Harbor / Marina,Tonkatsu Restaurant,...,Boutique,Historic Site,Clothing Store,Steakhouse,Bavarian Restaurant,Boat or Ferry,Coffee Shop,Scenic Lookout,Department Store,Museum
9,"Washington DC, USA",Garden,Outdoor Sculpture,Hotel,Bar,Monument / Landmark,Art Museum,Hotel Bar,American Restaurant,Sandwich Place,...,Gift Shop,Mediterranean Restaurant,Museum,Mexican Restaurant,Café,Spa,French Restaurant,Cocktail Bar,Nightlife Spot,Food Truck


In [51]:
city_merged.loc[city_merged['Cluster Labels'] == 3, city_merged.columns[[0] + list(range(4, city_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
18,"San Francisco/Oakland, USA",Auto Garage,Yoga Studio,Empanada Restaurant,Escape Room,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,...,Filipino Restaurant,Fish Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant


In [52]:
city_merged.loc[city_merged['Cluster Labels'] == 4, city_merged.columns[[0] + list(range(4, city_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
14,"Dallas/Fort Worth, USA",Seafood Restaurant,Mexican Restaurant,Hotel,Yoga Studio,Empanada Restaurant,Escape Room,Event Space,Exhibit,Fabric Shop,...,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck
